In [167]:
from icecream import ic
from abc import ABC, abstractmethod
import numpy as np
from numpy import linalg as LA
import graphviz

# Libraries
- $\texttt{forward\_backward\_functions\_and\_nodes}$: this library contains three things: 
    - The mathematical operators are defined as classes
    - The nodes of a node tree are defined by their behavior: variables/endnodes are defined as instances of $\texttt{Expr\_end\_node()}$, while functions are defined as instances of $\texttt{Expr\_node()}$. 
    - ABC-Function: Each time a class is called, it is defined as an instance of $\texttt{Expr\_node()}$. This means that calling an operator class is enough for the propagation algorithm to understand what and where the nodes are. 
- $\texttt{print\_graph}$

In [168]:
from forward_backward_functions_and_nodes import * 
from print_graph import print_graph

# Propagation algorithms
The forward pass gives the computated value of the specified node. If the node is not an endnode, it will keep recurring itself until it reaches one. \
The backward propagation is a recursive function that will record the argument of the outer derivative, compute the outer derivative, and mulitplty it with the derivative of the argument. The derivative of the argument is done the same way. The algorithm will keep doing this unit it reaches an endnode, where it will assign the value of the endnode with the total accumulated product. \
The possibility of multiple arguments is considered with the $\texttt{if len(node.childs) == 1}$ - clause.

In [169]:
def forward(node):
    return node.forward_func(*(forward(child) for child in node.childs)) if type(node) is not Expr_end_node else node.value 
    
# def backward(node, value = np.float64(1)):
#     if type(node) is not Expr_end_node:
#         child_values = [forward(child) for child in node.childs] # computes the argument of the outer derivative. In other words: it computes g of f'(g)
#         if len(node.childs) == 1:
#             new_value = node.backward_func(*child_values) # computes the outer derivative f'(g)
#             if value.ndim == 0 or new_value.ndim == 0:
#                 backward(node.childs[0], value * new_value)
#             else: 
#                 backward(node.childs[0], new_value.T @ value) # @ is matrix product
#         else:
#             for child, new_value in zip(node.childs, node.backward_func(*child_values), strict=True):
#                 if value.ndim == 0 or new_value.ndim == 0:
#                     backward(child, value * new_value)
#                 else: 
#                     backward(child, new_value.T @ value)                 
#     else:
#         node.grad_value += value


def backward(node, value = np.float64(1)):
    if type(node) is not Expr_end_node:
        child_values = [forward(child) for child in node.childs]
        if len(node.childs) == 1:
            # product of inner derivatives
            # value =parant_node derivative(all of parant's childs feeded forward)
            # new_value = derivative_of_current_node(all of current cilds feeded forward)
            new_value = node.backward_func(*child_values)
            if value.ndim == 0 or new_value.ndim == 0:
                backward(node.childs[0], value * new_value)
            else: 
                backward(node.childs[0], value @ new_value) # @ is matrix product
            
        else:
            for child, new_value in zip(node.childs, node.backward_func(*child_values), strict=True):
                if value.ndim == 0 or new_value.ndim == 0:
                    backward(child, value * new_value)
                else: 
                    backward(child, value @ new_value ) # @ is matrix product                  
    else: 
        node.grad_value += value

# Example functions

The following cells are structured as follows: 
- Each cell contains one example function. \
For each function, we define the operators, parameters, and the function itself. The operators are defined as their respective operator classes. \
Example: 
| mathematical operator | operator class  |
| --- | --- |
| + | Add() |
| $\cdot$ | Multiply()|
| sin() | Sin() | 

- This function will then be depicted as a node tree via the $\texttt{print\_graph}$ function.
- Finally, we perform the forward and backward propagation. The values of the propagations will be compared with values of the analytically solven function and derivative(s).

## Parameters and mathematical operators

In [170]:
x1 = Expr_end_node(np.random.rand(1))
x2 = Expr_end_node(np.random.rand(1))
x = Expr_end_node(np.random.rand(1))
w = Expr_end_node(np.random.rand(3,3))
xv = Expr_end_node(np.random.rand(3))
b = Expr_end_node(np.random.rand(3))

add = Add()
add_2 = Add_scalar(2)
multiply = Multiply()
multiply_3 = Multiply_scalar(3)
multiply_4 = Multiply_scalar(4)
sin = Sin()
log = Log()
tanh = Tanh()
vecadd = Vector_vector_sum()
matmul = Matrix_vector_product()

## Function 1: $f(x_1,x_2) = \log(x_1 \cdot x_2) \cdot \sin(x_2) $

In [171]:
# defining the function
func1 = multiply(log(multiply(x1, x2)), sin(x2))

# graphical depiction
graph1 = graphviz.Digraph('graph1', comment='test') 
graph1.attr(rankdir="LR")
print_graph(func1, graph1)
graph1.render(directory='graph_out/tt', view=True)


# analytical function and its derivative(s)
mfunc1 = np.log(x1.value * x2.value) * np.sin(x2.value) # analytical function
mdfunc1dx1 = np.sin(x2.value) / x1.value # analytical derivative w.r.t. x1 
mdfunc1dx2 = np.sin(x2.value) / x2.value + np.log(x1.value * x2.value) * np.cos(x2.value) # analytical derivative w.r.t. x2

# comparison to analytical value
# comparison 1: forward propagation
ic(forward(func1)) # value of the function via forward propagation
ic(np.log(x1.value * x2.value) * np.sin(x2.value)) # value of the analytical function

# comparison 2: backward propagation
# as we reuse the same parameter names for multiple functions, we set the derivatives w.r.t. the parameters to zero, before performing the derivatives.
x1.grad_value=0
x2.grad_value=0
backward(func1) # performing the derivative via backward propagation
ic(x1.grad_value) # value of the derivative w.r.t. x1 via backward propagation
ic(np.sin(x2.value) / x1.value) # value of the analytical derivative w.r.t. x1 
ic(x2.grad_value) # value of the derivative w.r.t. x2 via backward propagation
ic(np.sin(x2.value) / x2.value + np.log(x1.value * x2.value) * np.cos(x2.value)) # value of the analytical derivative w.r.t. x2


# Result
print("function 1")
print("Calculus values of x1 derivative and x2 derivative:", mdfunc1dx1, mdfunc1dx2 )
print("Compared to derivatives through chain rule:        ", x1.grad_value, x2.grad_value)

ic| forward(func1): array([-0.78560797])
ic| np.log(x1.value * x2.value) * np.sin(x2.value): array([-0.78560797])
ic| x1.grad_value: array([1.29875479])
ic| np.sin(x2.value) / x1.value: array([1.29875479])
ic| x2.grad_value: array([-1.66175517])
ic| np.sin(x2.value) / x2.value + np.log(x1.value * x2.value) * np.cos(x2.value): array([-1.66175517])


function 1
Calculus values of x1 derivative and x2 derivative: [1.29875479] [-1.66175517]
Compared to derivatives through chain rule:         [1.29875479] [-1.66175517]


## Function 2: $g(x_1, x_2) = x_1 \cdot x_2 (x_1 + x_2) $ 

In [172]:
# defining the function
func2 = multiply(x1, multiply(x2, add(x1, x2)))

# graphical depiction
graph2 = graphviz.Digraph('graph2', comment='test') 
graph2.attr(rankdir="LR")
print_graph(func2, graph2)
graph2.render(directory='graph_out/tt', view=True)

# analytical function and its derivative(s)
mfunc2 = x1.value * x2.value * (x1.value + x2.value)
mdfunc2dx1 = x2.value * (x1.value + x2.value) + x1.value * x2.value
mdfunc2dx2 = x1.value * (x1.value + x2.value) + x1.value * x2.value

# comparison
ic(forward(func2))
ic(x1.value * x2.value * (x1.value + x2.value))

x1.grad_value=0
x2.grad_value=0
backward(func2)
ic(x1.grad_value)
ic(mdfunc2dx1)
ic(x2.grad_value)
ic(mdfunc2dx2)

# result
print("Function 2")
print("Calculus values of x1 derivative and x2 derivative:", mdfunc2dx1, mdfunc2dx2 )
print("Compared to derivatives through chain rule:        ", x1.grad_value, x2.grad_value)

ic| forward(func2): array([0.03205012])
ic| x1.value * x2.value * (x1.value + x2.value): array([0.03205012])
ic| x1.grad_value: array([0.20950889])
ic| mdfunc2dx1: array([0.20950889])
ic| x2.grad_value: array([0.17428908])
ic| mdfunc2dx2: array([0.17428908])


Function 2
Calculus values of x1 derivative and x2 derivative: [0.20950889] [0.17428908]
Compared to derivatives through chain rule:         [0.20950889] [0.17428908]


## Function 3: $h(x) = 3x^2 + 4x + 2$

In [173]:
# defining the function
func3 = add_2( add( multiply_3(multiply(x,x)) , multiply_4(x) ))

# graphical depiction
graph3 = graphviz.Digraph('graph3', comment='test') 
graph3.attr(rankdir="LR")
print_graph(func3, graph3)
graph3.render(directory='graph_out/tt', view=True)

# comparison
mfunc3 = 3*x.value**2 + 4 * x.value + 2
mdfunc3dx = 6 * x.value + 4

ic(forward(func3))
ic(3*x.value**2 + 4 * x.value + 2)

x.grad_value = 0
backward(func3)
ic(mdfunc3dx)
ic(x.grad_value)


# Result
print("Function 3")
print("Calculus values of x derivative:                   ", mdfunc3dx)
print("Compared to derivatives through chain rule:        ", x.grad_value)

ic| forward(func3): array([2.12826373])
ic| 3*x.value**2 + 4 * x.value + 2: array([2.12826373])
ic| mdfunc3dx: array([4.18797859])
ic| x.grad_value: array([4.18797859])


Function 3
Calculus values of x derivative:                    [4.18797859]
Compared to derivatives through chain rule:         [4.18797859]


## Function 4: Neuron$(\vec x, w, \vec b) = \tanh(w\cdot \vec x + \vec b)$

### Manual algorithm
For the neuron activation function, we will compare the derivatives with what $\texttt{torch}$ computes. \
The backpropagation will include the loss function as the outermost derivative, so that we have
$$
 \frac{\partial \text{Loss}(\vec y)}{\partial \vartheta_{ij}} = \frac{\partial \text{Loss}}{\partial \vec y}  \frac{\partial \vec y}{\partial \vartheta_{ij}}
$$
with $\vartheta_{ij}$ either being the weight matrix $w_{ij}$ or the bias vector $\vec b$.

In [174]:
# loss_function   
def loss_function(y_pred, y_target):
     return (y_pred - y_target).pow(2).sum()
    
# (d loss)/(d y_pred) = 2*(y_pred - y_target)
def loss_backwards(y_pred, y_target):
     return 2*(y_pred - y_target)

In [175]:
w_value = np.random.rand(3,2)
xv_value = np.random.rand(2)
b_value = np.random.rand(3)
func4_target_value = np.random.rand(3)

w = Expr_end_node(w_value)
xv = Expr_end_node(xv_value)
b = Expr_end_node(b_value)
func4_target = Expr_end_node(func4_target_value)

tanh = Tanh()
vecadd = Vector_vector_sum()
matmul = Matrix_vector_product()

# defining the function
activation = Matrix_w_x_b()
func4 = tanh(activation(w,xv,b)) # = function 4

# prediction value for loss function
func4_predict = forward(func4)

# graphical depiction
graph4 = graphviz.Digraph('graph4', comment='test') 
graph4.attr(rankdir="LR")
print_graph(func4, graph4)
graph4.render(directory='graph_out/tt', view=True)


w.grad_value = np.float64(0)
xv.grad_value = np.float64(0)
b.grad_value = np.float64(0)
backward(func4)

grad_w = loss_backwards(func4_predict, func4_target.value)@w.grad_value
grad_xv = loss_backwards(func4_predict, func4_target.value)@xv.grad_value
grad_b = loss_backwards(func4_predict, func4_target.value)@b.grad_value

# reshaping the derivatives to the resepective parameter
grad_w_cs = grad_w.reshape(w.value.shape)
grad_xv_cs = grad_xv.reshape(xv.value.shape)
grad_b_cs = grad_b.reshape(b.value.shape)

ic(grad_w_cs)
ic(grad_xv_cs)
ic(grad_b_cs)


# Result
print("Function 4")
print("Derivatives through chain rule:        \n", "w :\n",
      grad_w_cs, "\n xv: \n",
      grad_xv_cs, "\n b: \n", 
      grad_b_cs)

ic| grad_w_cs: array([[ 0.17129803,  0.13099558],
                      [ 0.46455712,  0.35525761],
                      [-0.00722303, -0.00552362]])
ic| grad_xv_cs: array([0.40642053, 0.58518156])
ic| grad_b_cs: array([ 0.19586661,  0.53118667, -0.008259  ])


Function 4
Derivatives through chain rule:        
 w :
 [[ 0.17129803  0.13099558]
 [ 0.46455712  0.35525761]
 [-0.00722303 -0.00552362]] 
 xv: 
 [0.40642053 0.58518156] 
 b: 
 [ 0.19586661  0.53118667 -0.008259  ]


### Function 4 extension: multiple layers

In [176]:
w2_value = np.random.rand(3,3)
b2_value = np.random.rand(3)
func4_2_target_value = np.random.rand(3)

w2 = Expr_end_node(w2_value)
b2 = Expr_end_node(b2_value)
func4_2_target = Expr_end_node(func4_2_target_value)

# second neuron
func4_2 = tanh(activation(w2, func4, b2))

# graphical depiction
graph4_2 = graphviz.Digraph('graph4.2', comment='test') 
graph4_2.attr(rankdir="LR")
print_graph(func4_2, graph4_2)
graph4_2.render(directory='graph_out/tt', view=True)

w2.grad_value = np.float64(0)
b2.grad_value = np.float64(0)
w.grad_value = np.float64(0)
b.grad_value = np.float64(0)
backward(func4_2)

grad_w2 = loss_backwards(func4_2_predict, func4_2_target.value)@w2.grad_value
grad_b2 = loss_backwards(func4_2_predict, func4_2_target.value)@b2.grad_value

# reshaping the derivatives to the resepective parameter
grad_w2_rs = grad_w2.reshape(w2.value.shape)
grad_b2_rs = grad_b2.reshape(b2.value.shape)

ic(grad_w2_rs)
ic(grad_b2_rs)

ic| grad_w2_rs: array([[0.03194891, 0.02842431, 0.0328905 ],
                       [0.16102504, 0.14326079, 0.16577069],
                       [0.12138001, 0.10798939, 0.12495726]])
ic| grad_b2_rs: array([0.03484134, 0.1756031 , 0.13236889])


array([0.03484134, 0.1756031 , 0.13236889])

### Comparison to $\texttt{torch}$

In [177]:
import torch

In [178]:
# Same code as above, but the random parameters need to be in the same cell for comparison
# Everything inside the lined box is a repetition, skip to torch code
# -------------------------------------------------------------------------------------------
# actual values
w_value = np.random.rand(3,2)
xv_value = np.random.rand(2)
b_value = np.random.rand(3)

w2_value = np.random.rand(3,3)
b2_value = np.random.rand(3)
func4_2_target_value = np.random.rand(3)

# node expression
w = Expr_end_node(w_value)
xv = Expr_end_node(xv_value)
b = Expr_end_node(b_value)

w2 = Expr_end_node(w2_value)
b2 = Expr_end_node(b2_value)

func4_2_target = Expr_end_node(func4_2_target_value)


tanh = Tanh()
vecadd = Vector_vector_sum()
matmul = Matrix_vector_product()
activation = Matrix_w_x_b()

func4 = tanh(activation(w,xv,b))
func4_2 = tanh(activation(w2, func4, b2))
func4_2_predict = forward(func4_2)

w.grad_value = np.float64(0)
xv.grad_value = np.float64(0)
b.grad_value = np.float64(0)
backward(func4_2)

grad_w2 = loss_backwards(func4_2_predict, func4_2_target.value)@w2.grad_value
grad_b2 = loss_backwards(func4_2_predict, func4_2_target.value)@b2.grad_value

# reshaping the derivatives to the resepective parameter
grad_w2_rs = grad_w2.reshape(w2.value.shape)
grad_b2_rs = grad_b2.reshape(b2.value.shape)

# -------------------------------------------------------------------------------------------
# here starts the torch code
def forward_function(x, w1, b1, w2, b2):
     return torch.tanh(w2 @ torch.tanh(w1 @ x + b1) + b2)

# torch expression for parameters
xv_t = torch.tensor(xv_value, requires_grad=True)
w_t = torch.tensor(w_value, requires_grad=True)
b_t = torch.tensor(b_value, requires_grad=True)

w2_t = torch.tensor(w2_value, requires_grad=True)
b2_t = torch.tensor(b2_value, requires_grad=True)

func4_2_target_t = torch.tensor(func4_2_target_value)


func4_2_predict_t = forward_function(xv_t, w_t, b_t, w2_t, b2_t)
loss_t = loss_function(func4_2_predict_t, func4_2_target_t)

loss_t.backward()

ic(grad_w2_rs)
ic(grad_b2_rs)
ic(w2_t.grad)
ic(b2_t.grad)

print("Function 4")
print("torch values for w and b: \n",
      "w: \n", w2_t.grad, "\n",
      "b: \n", b2_t.grad, "\n")
print("compared to manual derivatives through chain rule: \n", 
      "w :\n", grad_w2_rs, "\n",
      "b: \n", grad_b2_rs)

ic| grad_w2_rs: array([[0.03448134, 0.03612454, 0.03667722],
                       [0.22805968, 0.23892778, 0.2425832 ],
                       [0.25727241, 0.26953263, 0.27365628]])
ic| grad_b2_rs: array([0.03730443, 0.24673157, 0.27833603])
ic| w2_t.grad: tensor([[0.0345, 0.0361, 0.0367],
                       [0.2281, 0.2389, 0.2426],
                       [0.2573, 0.2695, 0.2737]], dtype=torch.float64)
ic| b2_t.grad: tensor([0.0373, 0.2467, 0.2783], dtype=torch.float64)


Function 4
torch values for w and b: 
 w: 
 tensor([[0.0345, 0.0361, 0.0367],
        [0.2281, 0.2389, 0.2426],
        [0.2573, 0.2695, 0.2737]], dtype=torch.float64) 
 b: 
 tensor([0.0373, 0.2467, 0.2783], dtype=torch.float64) 

compared to manual derivatives through chain rule: 
 w :
 [[0.03448134 0.03612454 0.03667722]
 [0.22805968 0.23892778 0.2425832 ]
 [0.25727241 0.26953263 0.27365628]] 
 b: 
 [0.03730443 0.24673157 0.27833603]
